In [60]:
import spotipy
import json
import webbrowser
import urllib.request
import overpass
import spotipy.util as util
import geohash
import pygeohash as pgh

In [61]:
with open("geocode_keys.txt", "r") as geokey_file:
    geo_key = geokey_file.read()

In [96]:
address1 = "15 Elm St, Totowa, NJ, 07512, US"
address2 = "39 E George St, Providence, RI 02906, US"

nospace_address1 = (
    address1
        .replace(",", "")
        .replace(" ", "+")
)

nospace_address2 = (
    address2
        .replace(",", "")
        .replace(" ", "+")
)

# print(nospace_address1, nospace_address2)

map_url = f"https://geocode.maps.co/search?q={nospace_address2}&api_key={geo_key}"

map_request = urllib.request.Request(map_url)
map_response = urllib.request.urlopen(map_request)

map_data = json.loads(map_response.read())
lat = map_data[0]['lat']
lon = map_data[0]['lon']
print(lat, lon)

41.8256933 -71.3926077


In [97]:
test_hash = pgh.encode(float(lat), float(lon), precision=5)
print(test_hash)

drmjz


In [ ]:
with open("ticketmaster_keys.txt", "r") as key_file:
    ticketmaster_key = key_file.read()

with open("genreIDs.json", "r") as genreID_file:
    genre_list = json.loads(genreID_file.read())

# print(genre_list['rap'])

KnvZfZ7vAv1


In [98]:
geohash = "gcpuv"
unit = "km"
radius = 10
genre = genre_list['electronic']
url = f"https://app.ticketmaster.com/discovery/v2/events.json?size=20&radius={radius}&genreId={genre}&geoPoint={test_hash}&classificationName=music&apikey={ticketmaster_key}"

tm_request = urllib.request.Request(url)
tm_response = urllib.request.urlopen(tm_request)

full_event_listings = json.loads(tm_response.read())
event_names = full_event_listings['_embedded']['events']

print(len(event_names))


2


In [99]:
# if len(event_names) < 5:
#     radius*2
#     full_event_listings = json.loads(tm_response.read())

artist_names = [artist['name'] for artist in event_names]
artist_names_no_repeats = list(dict.fromkeys(artist_names))

print(artist_names_no_repeats)

['Tim Dahl, Makoto Kawabata, Harpy, Dull Care', 'Tightmare Before Christmas']


In [67]:
cred = "spotify_keys.json"
with open(cred, "r") as spotify_key_file:
    api_tokens = json.load(spotify_key_file)

spotify_key_file.close()

client_id = api_tokens['client_id']
client_secret = api_tokens['client_secret']
redirectURI = api_tokens['redirect']
username = api_tokens['username']

scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [ ]:
sp = spotipy.Spotify(auth=token)

playlist_songs = []

for artist in artist_names_no_repeats:
    returned_song = sp.search(q=artist, type="track", limit=1)

    if returned_song['tracks']['items'][0]['uri'] not in playlist_songs:
        playlist_songs.append(returned_song['tracks']['items'][0]['uri'])

# print(playlist_songs)

empty_playlist = sp.user_playlist_create(user=username, name=f"What's on in {address1.upper()}", public=True, description=f"songs from {genre} artists who have played live within a {radius+unit} radius of {address1.upper()}")
full_playlist = sp.user_playlist_add_tracks(username, empty_playlist['id'], playlist_songs)

In [69]:
webbrowser.open(empty_playlist['external_urls']['spotify'])

True